<a href="https://colab.research.google.com/github/flydragon2018/colab_notebooks/blob/main/mafat_baseline_model_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download training data



In [11]:
!pip -q install fsspec
!pip -q install gcsfs
import gcsfs
fs = gcsfs.GCSFileSystem()


In [ ]:
#copy train dataset from kaggle to colab

In [12]:
!gsutil -m cp -r  gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4   /content/ 

Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/metadata_train.csv...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10011_0_0.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10011_1280_0.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10011_2560_0.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10011_3840_0.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10011_6400_0.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10011_5120_0.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10135_3840_1280.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/10135_3840_2560.tiff...
Copying gs://kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/images/1013

In [34]:
!cp -rf /content/kds-eccd92be5a3a954d0795148c95aa80902d019d4c297f4d5c98c78df4/*  /content/

In [35]:
'''
#@title Click `Show code` in the code cell. { display-mode: "form" }

#dataset = "mini dataset" #@param ["full dataset", "mini dataset"]

dataset = "full dataset" #@param ["full dataset", "mini dataset"]


full_data = "https://drive.google.com/file/d/14MDE3RUpRMeUJ8VpI5BMFAZG-xnfon7B/view?usp=sharing"
mini_data = "https://drive.google.com/file/d/1PPVWxaiMsFALJBWUQIfkYZNFzhsv0gaa/view?usp=sharing"
url = mini_data
print('You selected', dataset)
if dataset == "mini dataset":
    url = mini_data
else:
    url = full_data
'''


'\n#@title Click `Show code` in the code cell. { display-mode: "form" }\n\n#dataset = "mini dataset" #@param ["full dataset", "mini dataset"]\n\ndataset = "full dataset" #@param ["full dataset", "mini dataset"]\n\n\nfull_data = "https://drive.google.com/file/d/14MDE3RUpRMeUJ8VpI5BMFAZG-xnfon7B/view?usp=sharing"\nmini_data = "https://drive.google.com/file/d/1PPVWxaiMsFALJBWUQIfkYZNFzhsv0gaa/view?usp=sharing"\nurl = mini_data\nprint(\'You selected\', dataset)\nif dataset == "mini dataset":\n    url = mini_data\nelse:\n    url = full_data\n'

In [36]:
#import gdown
#gdown.download(url=url, output="train_dataset.zip",quiet=False, fuzzy=True)

In [37]:
#!unzip '/content/train_dataset.zip'

## Data preparation

### modified spliting script based on the splitting script from mmrotate

In [38]:
%%writefile img_split.py
# Copyright (c) OpenMMLab. All rights reserved.
# Written by jbwang1997
# Reference: https://github.com/jbwang1997/BboxToolkit

import argparse
import codecs
import datetime
import itertools
import json
import logging
import os
import os.path as osp
import time
from functools import partial, reduce
from math import ceil
from multiprocessing import Manager, Pool

import cv2
import numpy as np
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

try:
    import shapely.geometry as shgeo
except ImportError:
    shgeo = None


def add_parser(parser):
    """Add arguments."""
    parser.add_argument(
        '--base-json',
        type=str,
        default=None,
        help='json config file for split images')
    parser.add_argument(
        '--nproc', type=int, default=10, help='the procession number')

    # argument for loading data
    parser.add_argument(
        '--img-dirs',
        nargs='+',
        type=str,
        default=None,
        help='images dirs, must give a value')
    parser.add_argument(
        '--ann-dirs',
        nargs='+',
        type=str,
        default=None,
        help='annotations dirs, optional')

    # argument for splitting image
    parser.add_argument(
        '--sizes',
        nargs='+',
        type=int,
        default=[1024],
        help='the sizes of sliding windows')
    parser.add_argument(
        '--gaps',
        nargs='+',
        type=int,
        default=[512],
        help='the steps of sliding widnows')
    parser.add_argument(
        '--rates',
        nargs='+',
        type=float,
        default=[1.],
        help='same as DOTA devkit rate, but only change windows size')
    parser.add_argument(
        '--img-rate-thr',
        type=float,
        default=0.6,
        help='the minimal rate of image in window and window')
    parser.add_argument(
        '--iof-thr',
        type=float,
        default=0.7,
        help='the minimal iof between a object and a window')
    parser.add_argument(
        '--no-padding',
        action='store_true',
        help='not padding patches in regular size')
    parser.add_argument(
        '--padding-value',
        nargs='+',
        type=int,
        default=[0],
        help='padding value, 1 or channel number')

    # argument for saving
    parser.add_argument(
        '--save-dir',
        type=str,
        default='.',
        help='to save pkl and split images')
    parser.add_argument(
        '--save-ext',
        type=str,
        default='.png',
        help='the extension of saving images')


def parse_args():
    """Parse arguments."""
    parser = argparse.ArgumentParser(description='Splitting images')
    add_parser(parser)
    args = parser.parse_args()

    if args.base_json is not None:
        with open(args.base_json, 'r') as f:
            prior_config = json.load(f)

        for action in parser._actions:
            if action.dest not in prior_config or \
                    not hasattr(action, 'default'):
                continue
            action.default = prior_config[action.dest]
        args = parser.parse_args()

    # assert arguments
    assert args.img_dirs is not None, "argument img_dirs can't be None"
    assert args.ann_dirs is None or len(args.ann_dirs) == len(args.img_dirs)
    assert len(args.sizes) == len(args.gaps)
    assert len(args.sizes) == 1 or len(args.rates) == 1
    assert args.save_ext in ['.png', '.jpg', 'bmp', '.tif', '.tiff']
    assert args.iof_thr >= 0 and args.iof_thr < 1
    assert args.iof_thr >= 0 and args.iof_thr <= 1
    assert not osp.exists(args.save_dir), \
        f'{osp.join(args.save_dir)} already exists'
    return args


def get_sliding_window(info, sizes, gaps, img_rate_thr):
    """Get sliding windows.

    Args:
        info (dict): Dict of image's width and height.
        sizes (list): List of window's sizes.
        gaps (list): List of window's gaps.
        img_rate_thr (float): Threshold of window area divided by image area.

    Returns:
        list[np.array]: Information of valid windows.
    """
    eps = 0.01
    windows = []
    width, height = info['width'], info['height']
    for size, gap in zip(sizes, gaps):
        assert size > gap, f'invaild size gap pair [{size} {gap}]'
        step = size - gap

        x_num = 1 if width <= size else ceil((width - size) / step + 1)
        x_start = [step * i for i in range(x_num)]
        if len(x_start) > 1 and x_start[-1] + size > width:
            x_start[-1] = width - size

        y_num = 1 if height <= size else ceil((height - size) / step + 1)
        y_start = [step * i for i in range(y_num)]
        if len(y_start) > 1 and y_start[-1] + size > height:
            y_start[-1] = height - size

        start = np.array(
            list(itertools.product(x_start, y_start)), dtype=np.int64)
        stop = start + size
        windows.append(np.concatenate([start, stop], axis=1))
    windows = np.concatenate(windows, axis=0)

    img_in_wins = windows.copy()
    img_in_wins[:, 0::2] = np.clip(img_in_wins[:, 0::2], 0, width)
    img_in_wins[:, 1::2] = np.clip(img_in_wins[:, 1::2], 0, height)
    img_areas = (img_in_wins[:, 2] - img_in_wins[:, 0]) * \
                (img_in_wins[:, 3] - img_in_wins[:, 1])
    win_areas = (windows[:, 2] - windows[:, 0]) * \
                (windows[:, 3] - windows[:, 1])
    img_rates = img_areas / win_areas
    if not (img_rates > img_rate_thr).any():
        max_rate = img_rates.max()
        img_rates[abs(img_rates - max_rate) < eps] = 1
    return windows[img_rates > img_rate_thr]


def poly2hbb(polys):
    """Convert polygons to horizontal bboxes.

    Args:
        polys (np.array): Polygons with shape (N, 8)

    Returns:
        np.array: Horizontal bboxes.
    """
    shape = polys.shape
    polys = polys.reshape(*shape[:-1], shape[-1] // 2, 2)
    lt_point = np.min(polys, axis=-2)
    rb_point = np.max(polys, axis=-2)
    return np.concatenate([lt_point, rb_point], axis=-1)


def bbox_overlaps_iof(bboxes1, bboxes2, eps=1e-6):
    """Compute bbox overlaps (iof).

    Args:
        bboxes1 (np.array): Horizontal bboxes1.
        bboxes2 (np.array): Horizontal bboxes2.
        eps (float, optional): Defaults to 1e-6.

    Returns:
        np.array: Overlaps.
    """
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]

    if rows * cols == 0:
        return np.zeros((rows, cols), dtype=np.float32)

    hbboxes1 = poly2hbb(bboxes1)
    hbboxes2 = bboxes2
    hbboxes1 = hbboxes1[:, None, :]
    lt = np.maximum(hbboxes1[..., :2], hbboxes2[..., :2])
    rb = np.minimum(hbboxes1[..., 2:], hbboxes2[..., 2:])
    wh = np.clip(rb - lt, 0, np.inf)
    h_overlaps = wh[..., 0] * wh[..., 1]

    l, t, r, b = [bboxes2[..., i] for i in range(4)]
    polys2 = np.stack([l, t, r, t, r, b, l, b], axis=-1)
    if shgeo is None:
        raise ImportError('Please run "pip install shapely" '
                          'to install shapely first.')
    sg_polys1 = [shgeo.Polygon(p) for p in bboxes1.reshape(rows, -1, 2)]
    sg_polys2 = [shgeo.Polygon(p) for p in polys2.reshape(cols, -1, 2)]
    overlaps = np.zeros(h_overlaps.shape)
    for p in zip(*np.nonzero(h_overlaps)):
        overlaps[p] = sg_polys1[p[0]].intersection(sg_polys2[p[-1]]).area
    unions = np.array([p.area for p in sg_polys1], dtype=np.float32)
    unions = unions[..., None]

    unions = np.clip(unions, eps, np.inf)
    outputs = overlaps / unions
    if outputs.ndim == 1:
        outputs = outputs[..., None]
    return outputs


def get_window_obj(info, windows, iof_thr):
    """

    Args:
        info (dict): Dict of bbox annotations.
        windows (np.array): information of sliding windows.
        iof_thr (float): Threshold of overlaps between bbox and window.

    Returns:
        list[dict]: List of bbox annotations of every window.
    """
    bboxes = info['ann']['bboxes']
    iofs = bbox_overlaps_iof(bboxes, windows)

    window_anns = []
    for i in range(windows.shape[0]):
        win_iofs = iofs[:, i]
        pos_inds = np.nonzero(win_iofs >= iof_thr)[0].tolist()

        win_ann = dict()
        for k, v in info['ann'].items():
            try:
                win_ann[k] = v[pos_inds]
            except TypeError:
                win_ann[k] = [v[i] for i in pos_inds]
        win_ann['trunc'] = win_iofs[pos_inds] < 1
        window_anns.append(win_ann)
    return window_anns


def crop_and_save_img(info, windows, window_anns, img_dir, no_padding,
                      padding_value, save_dir, anno_dir, img_ext):
    """

    Args:
        info (dict): Image's information.
        windows (np.array): information of sliding windows.
        window_anns (list[dict]): List of bbox annotations of every window.
        img_dir (str): Path of images.
        no_padding (bool): If True, no padding.
        padding_value (tuple[int|float]): Padding value.
        save_dir (str): Save filename.
        anno_dir (str): Annotation filename.
        img_ext (str): Picture suffix.

    Returns:
        list[dict]: Information of paths.
    """
    img = cv2.imread(osp.join(img_dir, info['filename']), -1)
    # img = Image.open(osp.join(img_dir, info['filename']))
    # img = np.asarray(img)
    patch_infos = []
    for i in range(windows.shape[0]):
        patch_info = dict()
        for k, v in info.items():
            if k not in ['id', 'fileanme', 'width', 'height', 'ann']:
                patch_info[k] = v

        window = windows[i]
        x_start, y_start, x_stop, y_stop = window.tolist()
        patch_info['x_start'] = x_start
        patch_info['y_start'] = y_start
        patch_info['id'] = \
            info['id'] + '__' + str(x_stop - x_start) + \
            '__' + str(x_start) + '___' + str(y_start)
        patch_info['ori_id'] = info['id']

        ann = window_anns[i]
        ann['bboxes'] = translate(ann['bboxes'], -x_start, -y_start)
        patch_info['ann'] = ann

        patch = img[y_start:y_stop, x_start:x_stop]
        if not no_padding:
            height = y_stop - y_start
            width = x_stop - x_start
            if height > patch.shape[0] or width > patch.shape[1]:
                padding_patch = np.empty((height, width, patch.shape[-1]),
                                         dtype=np.uint8)
                if not isinstance(padding_value, (int, float)):
                    assert len(padding_value) == patch.shape[-1]
                padding_patch[...] = padding_value
                padding_patch[:patch.shape[0], :patch.shape[1], ...] = patch
                patch = padding_patch
        patch_info['height'] = patch.shape[0]
        patch_info['width'] = patch.shape[1]

        cv2.imwrite(osp.join(save_dir, patch_info['id'] + img_ext), patch)
        patch_info['filename'] = patch_info['id'] + img_ext
        patch_infos.append(patch_info)

        bboxes_num = patch_info['ann']['bboxes'].shape[0]
        outdir = os.path.join(anno_dir, patch_info['id'] + '.txt')

        with codecs.open(outdir, 'w', 'utf-8') as f_out:
            if bboxes_num == 0:
                pass
            else:
                for idx in range(bboxes_num):
                    obj = patch_info['ann']
                    outline = ' '.join(list(map(str, obj['bboxes'][idx])))
                    diffs = str(
                        obj['diffs'][idx]) if not obj['trunc'][idx] else '2'
                    outline = outline + ' ' + obj['labels'][idx] + ' ' + diffs
                    f_out.write(outline + '\n')

    return patch_infos


def single_split(arguments, sizes, gaps, img_rate_thr, iof_thr, no_padding,
                 padding_value, save_dir, anno_dir, img_ext, lock, prog, total,
                 logger):
    """

    Args:
        arguments (object): Parameters.
        sizes (list): List of window's sizes.
        gaps (list): List of window's gaps.
        img_rate_thr (float): Threshold of window area divided by image area.
        iof_thr (float): Threshold of overlaps between bbox and window.
        no_padding (bool): If True, no padding.
        padding_value (tuple[int|float]): Padding value.
        save_dir (str): Save filename.
        anno_dir (str): Annotation filename.
        img_ext (str): Picture suffix.
        lock (object): Lock of Manager.
        prog (object): Progress of Manager.
        total (object): Length of infos.
        logger (object): Logger.

    Returns:
        list[dict]: Information of paths.
    """
    info, img_dir = arguments
    windows = get_sliding_window(info, sizes, gaps, img_rate_thr)
    window_anns = get_window_obj(info, windows, iof_thr)
    patch_infos = crop_and_save_img(info, windows, window_anns, img_dir,
                                    no_padding, padding_value, save_dir,
                                    anno_dir, img_ext)
    assert patch_infos

    lock.acquire()
    prog.value += 1
    msg = f'({prog.value / total:3.1%} {prog.value}:{total})'
    msg += ' - ' + f"Filename: {info['filename']}"
    msg += ' - ' + f"width: {info['width']:<5d}"
    msg += ' - ' + f"height: {info['height']:<5d}"
    msg += ' - ' + f"Objects: {len(info['ann']['bboxes']):<5d}"
    msg += ' - ' + f'Patches: {len(patch_infos)}'
    logger.info(msg)
    lock.release()

    return patch_infos


def setup_logger(log_path):
    """Setup logger.

    Args:
        log_path (str): Path of log.

    Returns:
        object: Logger.
    """
    logger = logging.getLogger('img split')
    formatter = logging.Formatter('%(asctime)s - %(message)s')
    now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    log_path = osp.join(log_path, now + '.log')
    handlers = [logging.StreamHandler(), logging.FileHandler(log_path, 'w')]

    for handler in handlers:
        handler.setFormatter(formatter)
        handler.setLevel(logging.INFO)
        logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger


def translate(bboxes, x, y):
    """Map bboxes from window coordinate back to original coordinate.

    Args:
        bboxes (np.array): bboxes with window coordinate.
        x (float): Deviation value of x-axis.
        y (float): Deviation value of y-axis

    Returns:
        np.array: bboxes with original coordinate.
    """
    dim = bboxes.shape[-1]
    translated = bboxes + np.array([x, y] * int(dim / 2), dtype=np.float32)
    return translated


def load_dota(img_dir, ann_dir=None, nproc=10):
    """Load DOTA dataset.

    Args:
        img_dir (str): Path of images.
        ann_dir (str): Path of annotations.
        nproc (int): number of processes.

    Returns:
        list: Dataset's contents.
    """
    assert osp.isdir(img_dir), f'The {img_dir} is not an existing dir!'
    assert ann_dir is None or osp.isdir(
        ann_dir), f'The {ann_dir} is not an existing dir!'

    print('Starting loading DOTA dataset information.')
    start_time = time.time()
    _load_func = partial(_load_dota_single, img_dir=img_dir, ann_dir=ann_dir)
    if nproc > 1:
        pool = Pool(nproc)
        contents = pool.map(_load_func, os.listdir(img_dir))
        pool.close()
    else:
        contents = list(map(_load_func, os.listdir(img_dir)))
    contents = [c for c in contents if c is not None]
    end_time = time.time()
    print(f'Finishing loading DOTA, get {len(contents)} iamges,',
          f'using {end_time - start_time:.3f}s.')

    return contents


def _load_dota_single(imgfile, img_dir, ann_dir):
    """Load DOTA's single image.

    Args:
        imgfile (str): Filename of single image.
        img_dir (str): Path of images.
        ann_dir (str): Path of annotations.

    Returns:
        dict: Content of single image.
    """
    img_id, ext = osp.splitext(imgfile)
    if ext not in ['.jpg', '.JPG', '.png', '.tif', '.bmp', '.tiff']:
        return None

    imgpath = osp.join(img_dir, imgfile)
    size = Image.open(imgpath).size
    txtfile = None if ann_dir is None else osp.join(ann_dir, img_id + '.txt')
    content = _load_dota_txt(txtfile)

    content.update(
        dict(width=size[0], height=size[1], filename=imgfile, id=img_id))
    return content


def _load_dota_txt(txtfile):
    """Load DOTA's txt annotation.

    Args:
        txtfile (str): Filename of single txt annotation.

    Returns:
        dict: Annotation of single image.
    """
    gsd, bboxes, labels, diffs = None, [], [], []
    if txtfile is None:
        pass
    elif not osp.isfile(txtfile):
        print(f"Can't find {txtfile}, treated as empty txtfile")
    else:
        with open(txtfile, 'r') as f:
            for line in f:
                if line.startswith('gsd'):
                    num = line.split(':')[-1]
                    try:
                        gsd = float(num)
                    except ValueError:
                        gsd = None
                    continue

                items = line.split(' ')
                if len(items) >= 9:
                    bboxes.append([float(i) for i in items[:8]])
                    labels.append(items[8])
                    diffs.append(int(items[9]) if len(items) == 10 else 0)

    bboxes = np.array(bboxes, dtype=np.float32) if bboxes else \
        np.zeros((0, 8), dtype=np.float32)
    diffs = np.array(diffs, dtype=np.int64) if diffs else \
        np.zeros((0,), dtype=np.int64)
    ann = dict(bboxes=bboxes, labels=labels, diffs=diffs)
    return dict(gsd=gsd, ann=ann)


def main():
    """Main function of image split."""
    args = parse_args()

    if args.ann_dirs is None:
        args.ann_dirs = [None for _ in range(len(args.img_dirs))]
    padding_value = args.padding_value[0] \
        if len(args.padding_value) == 1 else args.padding_value
    sizes, gaps = [], []
    for rate in args.rates:
        sizes += [int(size / rate) for size in args.sizes]
        gaps += [int(gap / rate) for gap in args.gaps]
    save_imgs = osp.join(args.save_dir, 'images')
    save_files = osp.join(args.save_dir, 'annfiles')
    os.makedirs(save_imgs)
    os.makedirs(save_files)
    logger = setup_logger(args.save_dir)

    print('Loading original data!!!')
    infos, img_dirs = [], []
    for img_dir, ann_dir in zip(args.img_dirs, args.ann_dirs):
        _infos = load_dota(img_dir=img_dir, ann_dir=ann_dir, nproc=args.nproc)
        _img_dirs = [img_dir for _ in range(len(_infos))]
        infos.extend(_infos)
        img_dirs.extend(_img_dirs)

    print('Start splitting images!!!')
    start = time.time()
    manager = Manager()
    worker = partial(
        single_split,
        sizes=sizes,
        gaps=gaps,
        img_rate_thr=args.img_rate_thr,
        iof_thr=args.iof_thr,
        no_padding=args.no_padding,
        padding_value=padding_value,
        save_dir=save_imgs,
        anno_dir=save_files,
        img_ext=args.save_ext,
        lock=manager.Lock(),
        prog=manager.Value('i', 0),
        total=len(infos),
        logger=logger)

    if args.nproc > 1:
        pool = Pool(args.nproc)
        patch_infos = pool.map(worker, zip(infos, img_dirs))
        pool.close()
    else:
        patch_infos = list(map(worker, zip(infos, img_dirs)))

    patch_infos = reduce(lambda x, y: x + y, patch_infos)
    stop = time.time()
    print(f'Finish splitting images in {int(stop - start)} second!!!')
    print(f'Total images number: {len(patch_infos)}')


if __name__ == '__main__':
    main()


Overwriting img_split.py


In [39]:
%%writefile split_config.json
{
    "nproc": 1,
    "img_dirs": [
      "images"
    ],
    "ann_dirs": [
      "labelTxt"
    ],
    "sizes": [
      640
    ],
    "gaps": [
      320
    ],
    "rates": [
      1.0
    ],
    "img_rate_thr": 0.6,
    "iof_thr": 0.7,
    "no_padding": false,
    "padding_value": [
      0,
      0,
      0
    ],
    "save_dir": "split_640_640",
    "save_ext": ".tiff"
  }

Overwriting split_config.json


### Split the frames to 640X640 size

In [40]:
!python img_split.py --base-json "split_config.json"

Traceback (most recent call last):
  File "img_split.py", line 595, in <module>
    main()
  File "img_split.py", line 538, in main
    args = parse_args()
  File "img_split.py", line 132, in parse_args
    assert not osp.exists(args.save_dir), \
AssertionError: split_640_640 already exists


### split dataset to train and validation sets

In [41]:
import numpy as np
import os
from random import sample
import shutil

In [42]:
data_root = './split_640_640'

In [43]:
images_paths = np.asarray(sorted(os.listdir(os.path.join(data_root, "images"))))
labels_paths = np.asarray(sorted(os.listdir(os.path.join(data_root, "annfiles"))))

l = len(images_paths) #length of data 
f = int(0.8 * l)  #number of elements you need
indices = sample(range(l),f)

images_paths_train = images_paths[indices]
labels_paths_train = labels_paths[indices]

images_paths_val = np.delete(images_paths, indices)
labels_paths_val = np.delete(labels_paths, indices)


In [44]:
#create folders with splits

# create folders
os.makedirs(os.path.join("train", "images"),exist_ok=True)
os.makedirs(os.path.join("train", "labelTxt"),exist_ok=True)

os.makedirs(os.path.join("val", "images"),exist_ok=True)
os.makedirs(os.path.join("val", "labelTxt"),exist_ok=True)

# move files to folders
for img_path, lbl_path in zip(images_paths_train, labels_paths_train):
    src = os.path.join(data_root, "images", img_path)
    dst = os.path.join("train", "images", img_path)
    shutil.copyfile(src, dst)
    
    src = os.path.join(data_root, "annfiles", lbl_path)
    dst = os.path.join("train", "labelTxt", lbl_path)
    shutil.copyfile(src, dst)
    
# move files to folders
for img_path, lbl_path in zip(images_paths_val, labels_paths_val):
    src = os.path.join(data_root, "images", img_path)
    dst = os.path.join("val", "images", img_path)
    shutil.copyfile(src, dst)
    
    src = os.path.join(data_root, "annfiles", lbl_path)
    dst = os.path.join("val", "labelTxt", lbl_path)
    shutil.copyfile(src, dst)

## Train model

### Install packages

In [45]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [46]:
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


In [47]:
# Install MMCV and MMDetection using MIM.
!pip3 install -U openmim
!mim install mmcv-full
!mim install mmdet

# Install MMRotate from the source.
!git clone https://github.com/open-mmlab/mmrotate.git
%cd mmrotate
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html
fatal: destination path 'mmrotate' already exists and is not an empty directory.
/content/mmrotate
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmrotate
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmrotate
    Found existing installation: mmrotate 0.3.4
    Can't uninstall 'mmrotate'. No files were found to uninstall.
  Running setup.py develop for mmrotate


In [48]:
import numpy as np
import os
import random
from random import sample
import shutil
import pandas as pd

In [49]:
from mmcv import collect_env
collect_env()

# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

0.3.4
2.28.1
11.6
GCC 9.3


### Download pre-trained weights and config

In [50]:
%cd ..

/content


In [51]:
# We use mim to download the pre-trained checkpoints for inference and finetuning.
!mim download mmrotate --config redet_re50_refpn_1x_dota_ms_rr_le90 --dest .

/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
processing redet_re50_refpn_1x_dota_ms_rr_le90...
redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth exists in /content
Successfully dumped redet_re50_refpn_1x_dota_ms_rr_le90.py to /content


### Define classes

In [52]:
# classes in dataset
CLASSES_ALL = ('small_vehicle','bus','medium_vehicle','large_vehicle',
     'double_trailer_truck','small_aircraft','large_aircraft','small_vessel','medium_vessel','large_vessel',
     'heavy_equipment', 'container','pylon', )

### Define custom dataset

In [53]:
from mmrotate.datasets.builder import ROTATED_DATASETS, PIPELINES
from mmrotate.datasets.dota import DOTADataset
import glob
import numpy as np
from mmrotate.core import poly2obb_np
import os

from mmdet.datasets.pipelines import LoadImageFromFile
from PIL import Image


@ROTATED_DATASETS.register_module()
class MAFATDataset(DOTADataset):
    """MAFAT dataset for detection."""

    CLASSES = CLASSES_ALL
    
    def load_annotations(self, ann_folder):
        """
            Args:
                ann_folder: folder that contains DOTA v1 annotations txt files
        """
        cls_map = {c: i
                   for i, c in enumerate(self.CLASSES)
                   }  # in mmdet v2.0 label is 0-based
        ann_files = glob.glob(ann_folder + '/*.txt')
        data_infos = []
        if not ann_files:  # test phase
            ann_files = glob.glob(ann_folder + '/*.tiff')
            for ann_file in ann_files:
                data_info = {}
                img_id = osp.split(ann_file)[1][:-4]
                img_name = img_id + '.tiff'
                data_info['filename'] = img_name
                data_info['ann'] = {}
                data_info['ann']['bboxes'] = []
                data_info['ann']['labels'] = []
                data_infos.append(data_info)
        else:
            for ann_file in ann_files:
                data_info = {}
                img_id = osp.split(ann_file)[1][:-4]
                img_name = img_id + '.tiff'
                data_info['filename'] = img_name
                data_info['ann'] = {}
                gt_bboxes = []
                gt_labels = []
                gt_polygons = []
                gt_bboxes_ignore = []
                gt_labels_ignore = []
                gt_polygons_ignore = []

                if os.path.getsize(ann_file) == 0 and self.filter_empty_gt:
                    continue

                with open(ann_file) as f:
                    s = f.readlines()
                    for si in s:
                        bbox_info = si.split()
                        poly = np.array(bbox_info[:8], dtype=np.float32)
                        try:
                            x, y, w, h, a = poly2obb_np(poly, self.version)
                        except:  # noqa: E722
                            continue
                        cls_name = bbox_info[8]
                        # difficulty = int(bbox_info[9])
                        difficulty = 0
                        label = cls_map[cls_name]
                        if difficulty > self.difficulty:
                            pass
                        else:
                            gt_bboxes.append([x, y, w, h, a])
                            gt_labels.append(label)
                            gt_polygons.append(poly)

                if gt_bboxes:
                    data_info['ann']['bboxes'] = np.array(
                        gt_bboxes, dtype=np.float32)
                    data_info['ann']['labels'] = np.array(
                        gt_labels, dtype=np.int64)
                    data_info['ann']['polygons'] = np.array(
                        gt_polygons, dtype=np.float32)
                else:
                    data_info['ann']['bboxes'] = np.zeros((0, 5),
                                                          dtype=np.float32)
                    data_info['ann']['labels'] = np.array([], dtype=np.int64)
                    data_info['ann']['polygons'] = np.zeros((0, 8),
                                                            dtype=np.float32)

                if gt_polygons_ignore:
                    data_info['ann']['bboxes_ignore'] = np.array(
                        gt_bboxes_ignore, dtype=np.float32)
                    data_info['ann']['labels_ignore'] = np.array(
                        gt_labels_ignore, dtype=np.int64)
                    data_info['ann']['polygons_ignore'] = np.array(
                        gt_polygons_ignore, dtype=np.float32)
                else:
                    data_info['ann']['bboxes_ignore'] = np.zeros(
                        (0, 5), dtype=np.float32)
                    data_info['ann']['labels_ignore'] = np.array(
                        [], dtype=np.int64)
                    data_info['ann']['polygons_ignore'] = np.zeros(
                        (0, 8), dtype=np.float32)

                data_infos.append(data_info)

        self.img_ids = [*map(lambda x: x['filename'][:-4], data_infos)]
        return data_infos
    
   
@PIPELINES.register_module()
class LoadImageFromFilePIL(LoadImageFromFile):
    """Load an image from file using PIL Image.
        Images are 16 bit grayscale tiff files, stacked to a 3 channel to fit the DNN"""

    def __call__(self, results):
        """Call functions to load image and get image meta information.

        Args:
            results (dict): Result dict from :obj:`mmdet.CustomDataset`.

        Returns:
            dict: The dict contains loaded image and meta information.
        """

        if self.file_client is None:
            self.file_client = mmcv.FileClient(**self.file_client_args)

        if results['img_prefix'] is not None:
            filename = osp.join(results['img_prefix'],
                                results['img_info']['filename'])
        else:
            filename = results['img_info']['filename']

        img = cv2.imread(filename, -1)
        img = np.stack((img,)*3, axis=-1)
        if self.to_float32:
            img = img.astype(np.float32)

        results['filename'] = filename
        results['ori_filename'] = results['img_info']['filename']
        results['img'] = img
        results['img_shape'] = img.shape
        results['ori_shape'] = img.shape
        results['img_fields'] = ['img']
        return results

KeyError: ignored

### Modify the config

In [ ]:
from mmcv import Config
cfg = Config.fromfile('redet_re50_refpn_1x_dota_ms_rr_le90.py')

In [ ]:
from mmdet.apis import set_random_seed
import os

# # Modify dataset type and path
cfg.dataset_type = 'MAFATDataset'
cfg.data_root = './'

cfg.img_norm_cfg = dict(type='Normalize',
    mean=[795.0878, 795.0878, 795.0878], std=[460.0955, 460.0955, 460.0955], to_rgb=True)

cfg.data.train.type = 'MAFATDataset'
cfg.data.train.data_root = os.path.join(cfg.data_root, 'train')
cfg.data.train.ann_file = 'labelTxt'
cfg.data.train.img_prefix = 'images'
color_type='unchanged'

cfg.data.test.type = 'MAFATDataset'
cfg.data.test.data_root = os.path.join(cfg.data_root, 'val')
cfg.data.test.ann_file = 'labelTxt'
cfg.data.test.img_prefix = 'images'

cfg.data.val.type = 'MAFATDataset'
cfg.data.val.data_root = os.path.join(cfg.data_root, 'val')
cfg.data.val.ann_file = 'labelTxt'
cfg.data.val.img_prefix = 'images'

# edit pipeline to handle 16bit images
cfg.train_pipeline[0].color_type = 'unchanged'
cfg.train_pipeline[0].type = 'LoadImageFromFilePIL'
cfg.test_pipeline[0].color_type = 'unchanged'
cfg.test_pipeline[0].type = 'LoadImageFromFilePIL'

cfg.data.train.pipeline[5] = cfg.img_norm_cfg
cfg.data.val.pipeline[1]['transforms'][1] = cfg.img_norm_cfg
cfg.data.test.pipeline[1]['transforms'][1] = cfg.img_norm_cfg

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head[0].num_classes = len(CLASSES_ALL)
cfg.model.roi_head.bbox_head[1].num_classes = len(CLASSES_ALL)
# Load pre-trainied weights, trained on DOTA
cfg.load_from = 'redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

cfg.optimizer.lr = 1e-3
cfg.lr_config.warmup = None
#cfg.runner.max_epochs = 4
#cfg.log_config.interval = 2

cfg.runner.max_epochs = 12
cfg.log_config.interval = 2

cfg.data.samples_per_gpu=2
cfg.data.workers_per_gpu=2

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 2
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'

# We can also use tensorboard to log the training process
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='TensorboardLoggerHook')]

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
# save edited config file 
dump_file = "my_config_redet.py"
cfg.dump(dump_file)

### Train a new detector

In [ ]:
from mmcv import collect_env
import mmcv
collect_env()

# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import os.path as osp

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import cv2

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=False)
# train_detector(model, datasets, cfg, distributed=False, validate=True) # doesn't work in colab

## Deploy a model for submission

###Create model file

In [ ]:
%%writefile model.py
import mmcv
from mmcv.runner import load_checkpoint

from mmrotate.apis import inference_detector_by_patches
from mmrotate.models import build_detector

import os
import math
import numpy as np
from zipfile import ZipFile

class model():
    def __init__(self):
        self.make_initial_pass_over_dataset = True
        self.CLASSES = ('small_vehicle','bus','medium_vehicle','large_vehicle',
     'double_trailer_truck','small_aircraft','large_aircraft','small_vessel','medium_vessel','large_vessel',
     'heavy_equipment', 'container','pylon', ) # MAFAT classes
        self.stats = []
        self.checkpoint_path = r"latest.pth"
        self.config_path = r"my_config_redet.py"
        # Initialize variables to store the running sum and square sum of pixel values
        self.running_sum = 0
        self.running_square_sum = 0
        # Initialize a variable to store the number of images processed
        self.num_images = 0
    
    def collect_statistics_iter(self, img, meta):
        '''img is a grayscale 16bit image of size 1280x1280 pixels (as numpy array).
        
        
        metadata is a dictionary with the metadata for the given image
        (similar to training phase, but without AOI and Hermetic data)
        '''
        # you can do something with the meta data
        
        # calculate mean and std using welford method        
        self.mean_and_std(img) 
        
        
        
    def update_model(self):
        # get mean and std
        self.std *= 2 ** 16
        self.mean *= 2 ** 16
        
        print("mean: %f, std: %f" % (self.mean, self.std))
        self.stats.append(self.mean)
        self.stats.append(self.std)
        
        # update test pipeline with computed statistics
        self.model.cfg.data.test.pipeline[1]['transforms'][1]['mean'] = [self.stats[0]]*3
        self.model.cfg.data.test.pipeline[1]['transforms'][1]['std'] = [self.stats[1]]*3
        
    
    def mean_and_std(self, image):
        
        # Update the running sum and square sum with the new image data
        image = image / 2 ** 16
        self.running_sum += image.sum()
        self.running_square_sum += (image ** 2).sum()

        # Increment the number of images processed
        self.num_images += 1

        # Compute the mean and standard deviation on the fly
        self.mean = self.running_sum / (self.num_images * image.shape[0] * image.shape[1])
        self.std = np.sqrt((self.running_square_sum / (self.num_images * image.shape[0] * image.shape[1])) - self.mean**2)
        
 
    
    
    def load(self, dir_path):
        """loads the model.
        dir_path is for internal use only - do not remove it.
        all other paths, such as self.config_path should only contain the file name (in this case: my_config_redet.py).
        these paths must be concatenated with dir_path - see example in the code below
        make sure these files are in the same directory as the model.py file

        Args:
            dir_path (string): path to the submission directory (for internal use only).
        """        
        # check if directory contains a zip file, i.e. the model weights were downloaded from google drive
        files = os.listdir(dir_path)
        for filename in files:
            if filename.endswith("zip"):
                with ZipFile(os.path.join(dir_path, filename), 'r') as z:
                    z.extractall(dir_path)
                break
            
        # join paths
        config_path = os.path.join(dir_path, self.config_path)
        checkpoint_path = os.path.join(dir_path, self.checkpoint_path)
        # Set the device to be used for evaluation
        self.device='cuda:0'

        # Load the config
        self.config = mmcv.Config.fromfile(config_path)
        # adjust number of classes
        # self.config.model.bbox_head.num_classes = len(CLASSES_ALL)
        # Set pretrained to be None since we do not need pretrained model here
        self.config.model.pretrained = None

        # Initialize the detector
        self.model = build_detector(self.config.model)

        # Load checkpoint
        self.checkpoint = load_checkpoint(self.model, checkpoint_path, map_location=self.device)

        # Set the classes of models for inference
        self.model.CLASSES = self.checkpoint['meta']['CLASSES']

        # We need to set the model's cfg for inference
        self.model.cfg = self.config

        # Convert the model to GPU
        self.model.to(self.device)
        # Convert the model into evaluation mode
        self.model.eval()
        
    def predict(self, img, metadata=None):
        '''img is a grayscale 16bit image of size 1280x1280 pixels.
            expected output is a list of lists. each sub list represents all the detections for a class.
            the sub lists need to be in the same order as self.CLASSES.
            the detections need to be in the form of:
            confidence x1 y1 x2 y2 x3 y3 x4 y4
            
            metadata is a dictionary with the metadata for the given image
            (similar to training phase, but without AOI and Hermetic data)
        '''
            
        # the DNN expects a RGB image, we duplicate the grayscale image 3 times
        img = np.stack((img, )*3, axis=-1)
        # pass the image through the model to get detections
        result = inference_detector_by_patches(self.model, img, [640], [320], [1.0], 0.3)
        # result = inference_detector(self.model, img)
        for i, res in enumerate(result):
            result[i] = self.convert_rbb2polygon(res)
        return result
    
    def convert_rbb2polygon(self, bboxes):
        polygons = []
        for i, bbox in enumerate(bboxes):
            xc, yc, w, h, ag, conf = bbox
            wx, wy = w / 2 * np.cos(ag), w / 2 * np.sin(ag)
            hx, hy = -h / 2 * np.sin(ag), h / 2 * np.cos(ag)
            p1 = (xc - wx - hx, yc - wy - hy)
            p2 = (xc + wx - hx, yc + wy - hy)
            p3 = (xc + wx + hx, yc + wy + hy)
            p4 = (xc - wx + hx, yc - wy + hy)
            poly = np.array([conf, p1[0], p1[1], p2[0], p2[1], p3[0], p3[1], p4[0], p4[1]])
            polygons.append(poly)
        return polygons
        

###create metadata file

In [ ]:
%%writefile metadata
command: python3 $program/model.py $input $output

### Option 1 - weights size is small (total submission size under 300Mb)

In [ ]:
!cp tutorial_exps/epoch_4.pth ./latest.pth

In [ ]:
!zip -r submission_regular.zip model.py my_config_redet.py latest.pth metadata

###Option 2 - weights size is large (total submission size over 300Mb and under 1 Gb)

In [ ]:
!zip -r weights.zip latest.pth

In [ ]:
%%writefile model_url.txt
https://drive.google.com/file/d/1Y-L9CLqN7WVdm1nEdu6Lt7okyoCbjbx4/view?usp=sharing

In [ ]:
!zip -r submission_url.zip model.py my_config_redet.py model_url.txt metadata